# Convolutional Neural Networks for Handwritten Digit Recognition

Object recognition is one of the most exciting tasks in Computer Vision. The goal is to detect and identify objects in an image. The most popular dataset in object recognition involves recognizing handwritten digits. In this part of the lab, we will develop a convolutional neural network for classifying handwritten digits using the [Keras](https://keras.io/) library. We will evaluate the neural network on the well-known [MNIST dataset](http://yann.lecun.com/exdb/mnist/). The dataset was created by Yann LeCun, Corinna Cortes and Christopher Burges for evaluating machine learning models. Each instance corresponds to the image of a digit taken from a scanned document. Each image is a 28 by 28 pixel square (784 pixels total), and the digits are normalized in size and centred. There are 10 digits in total (0 to 9). Hence, there are 10 classes in total. The dataset is spit into a training set consisting of 60,000 images and a test set of 10,000 images.

Keras provides a function for directly loading the MNIST dataset. The dataset is downloaded automatically the first time this function is called and is stored in the disk. Run the following code to load the MNIST dataset. Then, use the ``show`` function (already implemented) to visualize a digit of the training set.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.datasets import mnist

def show(image):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap='gray')
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


#your code here

To reduce training time, we will carry out experiments only on a subset of the dataset. Specifically, the first 10,000 instances of the training set and the first 1,000 instances of the test set will serve as our new training and test sets, respectively.

In [ ]:
X_train = X_train[:10000,:,:]
y_train = y_train[:10000]
X_test = X_test[:1000,:,:]
y_test = y_test[:1000]

print("Shape of training matrix:", X_train.shape)
print("Shape of test matrix:", X_test.shape)

After loading the MNIST dataset and reducing its size, it is necessary to reshape all the instances so that their shape is the one a CNN would expect. In Keras, the layers used for two-dimensional convolutions expect the depth of the input image along with its dimensions. In the case of full-color images, the depth is equal to 3 and each dimension corresponds to its red, green and blue components. The images contained in the MNIST dataset are greyscale, hence, the depth is equal to 1. Therefore, to be able to apply two-dimensional convolutions, it is necessary to transform the shape of each image from (width, height) to (width, height, depth). Make use of the [reshape](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.reshape.html) function of NumPy to add an extra dimension to the training and test matrices.

In [ ]:
#your code here

The final preprocessing step is to convert the type of the images to float32, normalize their values to the range [0,1] and to encode the class labels using a one-hot scheme.

In [ ]:
from tensorflow.keras.utils import to_categorical

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Shape of training matrix:", X_train.shape)
print("Shape of test matrix:", X_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Upon completing the preprocessing pipeline, we can now start developing the [convolutional neural network](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN) architecture. Keras provides rich functionality for building CNNs since it offers various methods for creating convolutional and pooling layers. We first initialize a Sequential model.

In [ ]:
from tensorflow.keras.models import Sequential

#your code here

We next add a two-dimensional convolution layer to our model. This layer will create a convolution kernel that is convolved with the layer input to produce a tensor of outputs. Use the [`Conv2D`](https://keras.io/layers/convolutional/#conv2d) method of Keras to generate a convolution layer with 32 filters of size (3,3) and a ReLU activation function.

In [ ]:
from tensorflow.keras.layers import Conv2D

#your code here

Add a second convolution layer to the model. Set the number of filters to 64 and their size to (2,2). As in the case of the first layer use a ReLU activation function.

In [ ]:
#your code here

Then, define a pooling layer using the [`MaxPooling2D`](https://keras.io/layers/pooling/#maxpooling2d) method of Keras. Max pooling reduces the number of parameters in the model by sliding a 2x2 pooling filter on the output of the previous layer and taking the max of the 4 values in the 2x2 filter.

In [ ]:
from tensorflow.keras.layers import MaxPooling2D

#your code here

The next layer is a regularization layer using dropout. Use the [`Dropout`](https://keras.io/layers/core/#dropout) method of Keras to randomly exclude 20% of the neurons in the layer. Dropout has been shown to reduce overfitting.

In [ ]:
from tensorflow.keras.layers import Dropout

#your code here

We next transform the two-dimensional matrix that has emerged from the previous layers to a vector using the [`Flatten`](https://keras.io/layers/core/#flatten) function of Keras. This vector will serve as the input to a standard feedforward neural network. We also create a fully connected layer with 128 neurons and a ReLU activation function. Finally, we add an output layer consisting of 10 neurons for the 10 classes along with a softmax activation function to output probability-like predictions for each class.

In [ ]:
from tensorflow.keras.layers import Dense, Flatten

#your code here

We next compile the model and by declaring the loss function and the optimizer.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

We next print a summary representation of the model we generated. 

In [ ]:
model.summary()

We can now train the neural network and use it to make predictions. Train the model for 10 epochs. Set the batch size to 64. Use the test data as the validation dataset. Once the training has finished, evaluate the model on the test set and print the classification accuracy. Note that training may take several minutes. You can reduce the training time by running the code on a GPU instead of CPU.

In [ ]:
epochs = 10
batch_size = 64

#your code here

The first layer of the neural network we implemented is a convolution layer. This convolution layer is applied directly to the greyscale images of the MNIST dataset, hence, an interesting task is to visualize what these filters have learned. Next, we apply the 32 filters to an image of the training set and we visualize the 26x26 output of each filter.

In [ ]:
from tensorflow.keras import backend as K

def visualize(layer, img):
    convout1_f = K.function(model.inputs, [layer.output])

    convolutions = convout1_f(img)
    convolutions = np.squeeze(convolutions)

    print ('Shape of conv:', convolutions.shape)
    
    n = convolutions.shape[2]
    n = int(np.ceil(np.sqrt(n)))
    
    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(12,8))
    for i in range(convolutions.shape[2]):
        ax = fig.add_subplot(n,n,i+1)
        ax.imshow(convolutions[:,:,i], cmap='gray')
        
# choose an image
image = X_train[12,:,:]

# Keras requires the image to be in 4D
image = np.expand_dims(image, axis=0)
        
# Specify the layer to want to visualize
convout = model.layers[0]
visualize(convout, image)

We will next experiment with the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset which consists of 60,000 colour images of size 32 x 32. There are 10 classes in total, with 6,000 images per class. The dataset is split into a training and a test set consisting of 50,000 and 10,000 images, respectively. The 10 classes correspond to different objects such as airplanes, frogs and trucks.

We will first load the dataset and then implement a CNN to learn to classify the images into their classes.


In [ ]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

def show(image):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')

#your code here    

We will next use a dataset containing several thousand images of cats and dogs. These are colour images of size 160 x 160. This is a binary classification dataset. We will first load the dataset and visualize the first image of the training set.

In [ ]:
from tensorflow.keras.utils import get_file, image_dataset_from_directory

path_to_zip = get_file('cats_and_dogs.zip', origin='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip', extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=64, image_size=(160, 160))
validation_dataset = image_dataset_from_directory(validation_dir, shuffle=True, batch_size=64, image_size=(160, 160))

for images, labels in train_dataset.take(1):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(images[0].numpy().astype("uint8"))
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')

We will next implement a CNN to classify the images of the datasets. You can use the [Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Rescaling) layer of Keras to rescales input values to a new range (e.g., values between 0 and 1). 

In [ ]:
from tensorflow.keras.layers import Rescaling

model = Sequential()

#your code here

model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)

We will next use transfer learning to achieve higher performance on the dataset. Specifically, we will use the pretrained MobileNetV2 architecture to produce vector representation of the input images and then feed them to a trainable fully-connected layer to produce the output.

In [ ]:
from tensorflow.keras.applications import MobileNetV2

pretrained_model = MobileNetV2(input_shape=[160, 160, 3], include_top=False)
pretrained_model.trainable = False

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())

#your code here

model.summary()

 Since MobileNetV2 was trained on large datasets, we expect it to lead our model to higher validation accuracy. Train the model we created above. What do you observe?

In [ ]:
#your code here